In [1]:
%load_ext bigdata

In [2]:
%hive_start

In [3]:
%timeout 300

In [4]:
!hadoop fs -rm -f -r tmp;

In [5]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

-rw-r--r--   1 root supergroup       1355 2020-01-20 04:17 /tmp/output/data.tsv


In [6]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 13.77 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.233 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 2.145 seconds


In [7]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
SELECT
    concat(c1,',',
    size(c2),',',
    size(c3))
FROM
    t0;

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 0.013 seconds
CREATE TABLE salida
AS
SELECT
    concat(c1,',',
    size(c2),',',
    size(c3))
FROM
    t0;
Query ID = root_20200120041815_c9349fce-d03e-4a4f-b54f-cb87f75c8bee
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579493677098_0001, Tracking URL = http://5ac91a71ce8c:8088/proxy/application_1579493677098_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579493677098_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 04:18:35,304 Stage-1 map = 0%,  reduce = 0%
2020-01-20 04:18:43,308 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.11 sec
MapReduce Total cumulative CPU time: 4 seconds 110 msec
Ended Job = job_1579493677098_0001
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtere

In [8]:
%%hive
DROP TABLE IF EXISTS tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.022 seconds


In [9]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120041855_68edb12a-3a4b-47e0-af2b-d4dc2bc4bba6
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579493677098_0002, Tracking URL = http://5ac91a71ce8c:8088/proxy/application_1579493677098_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579493677098_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 04:19:06,847 Stage-1 map = 0%,  reduce = 0%
2020-01-20 04:19:13,849 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.0 sec
MapReduce Total cumulative CPU time: 2 seconds 0 msec
Ended Job = job_1579493677098_0002
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000

In [10]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [11]:
%hive_quit